## Geospatial Analysis of Sacramento Region for new Pizzeria

### Week 5 Assignment: Battle of the Neighborhoods

My clients are moving from the east coast back to Sacramento, CA to be close with their extended family. They currently run a successful Pizzeria in New Jersey which they will be selling to fund a new restaurant in the Sacramento region. They have hired my consulting firm to provide assistance to help them determine what part of the region would be the ideal spot to open their new restaurant. Optimally, they would like to open their restaurant in a region that is near event venues, corporate centers and recreational activities while also minimizing competition by opening in an area that lacks in current Pizzeria restaurants.  They really love downtown Sacramento, but they also hear the entire region is growing and there may be great opportunities within a 20 mile radius of the city.

- __Part 1 explore locations and venues within the City of Sacramento__   
   



- __Part 2 explore regions within 20 miles of Sacramento__
    - Look at U.S. Consensus data for top 20 most populated cities near Sacramento
    - Explore locations and venues within these top 20 cities

# __PART 1: THE CITY OF SACRAMENTO__

In [ ]:
import itertools
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from urllib.request import urlopen
from bs4 import BeautifulSoup
import numpy as np
import json 
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!pip install folium
import folium
import matplotlib.pyplot as plt
from matplotlib import pyplot
%matplotlib inline
import seaborn as sns

print('Libraries imported.')

## Let's Explore the City of Sacramento First 
- I identified a website that defined the __13 Top-Rated Tourist Attractions__ & Things to Do in the city of Sacramento, CA
    - URL: https://www.planetware.com/tourist-attractions-/sacramento-us-ca-sc.htm   
   


- I thought these __ATTRACTIONS__ would be a great starting point in our investigation for my clients:
    - Explore the Area around each one of these top Attractions
    - Group the neaby venues into categories and rank them based on their popularity (most frequented)
    - perform a cluster analysis using K-means clustering and try to get additional insight on venue categories 

In [ ]:
# Top 13 Top-Rated Tourist Attractions website, read with BeautifulSoup

url = urlopen(r"https://www.planetware.com/tourist-attractions-/sacramento-us-ca-sc.htm")
soup = BeautifulSoup(url, 'lxml')
soup

In [ ]:
#Scrape Attraction Names from website

Attraction_Name = []

for div in soup.find_all('div', class_='article_block site'):
    Attraction_Name.append(div.find_all('h2')[0].get_text().strip('\n'))

for i in range(len(Attraction_Name)):
    Attraction_Name[i]= Attraction_Name[i][4:].strip(',')

for i in range(len(Attraction_Name)):
    if i == 9 or i == 10 or i == 11:
        Attraction_Name[i] = Attraction_Name[i][1:]
    else:
        Attraction_Name[i] = Attraction_Name[i][0:]
        
Attraction_Name = Attraction_Name[:-2]
Attraction_Name

In [ ]:
#Some of the attractions failed my search to geolocator. So as my loop failed, I identified and had to update the culprits with code
#These are the Attractions that required an updated name before the geolocator identified the long, Lat coordinates

Attraction_Name[1]= 'Capital Park Sacramento'
Attraction_Name[2]= 'Sacramento Crocker Museum'
Attraction_Name[8]= 'Cathedral of the Blessed Sacrament Sacramento'
Attraction_Name[9]= 'California State Indian Museum'
Attraction_Name[10]= 'Midtown 1915 O St, Sacramento, CA 95811'
Attraction_Name[11]= 'American River Bike Trail'
Attraction_Name

In [ ]:
type(Attraction_Name)

In [ ]:
# Used geolocator and used a loop to search for, intentify longitude and latitude, then populate a dataframe
Latitude = []
Longitude = []

for name in Attraction_Name:
    geolocator = Nominatim(user_agent="foursquare_agent")
    location = geolocator.geocode(name)
    latitude = location.latitude
    longitude = location.longitude
    print(f'{name}: [{latitude}, {longitude}]')
    Latitude.append(latitude)
    Longitude.append(longitude)        
        
print('\n')
print('DataFrames Latitude & Longitude have been updated.')

In [ ]:
#fix lat long coordinates for Capital Park since geolocator came up with a location in Phillipines

Latitude[1] = float(38.57605)
Longitude[1] = float(-121.49072)
print(Latitude[1])
print(Longitude[1])

In [ ]:
#Time to merge the three seperate lists into one
Attractions = pd.DataFrame({'Attraction':Attraction_Name, 'Latitude':Latitude, 'Longitude': Longitude})
Attractions

In [ ]:
Attractions.shape

## Use location coordinates to explore Attractions on an interactive map

In [ ]:
# Identify the Latitude, Longitude for Sacramento, CA

address = 'Sacramento, CA'

geolocator = Nominatim(user_agent="coursera")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates for Sacramento are {}, {}.'.format(latitude, longitude))

In [ ]:
# Create a general map of Sacramento
map_sac = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, label in zip(Attractions['Latitude'], Attractions['Longitude'], Attractions['Attraction']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=7,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_sac)  
    
map_sac

In [ ]:
# Lets explore the venues that surround each of these Attractions
# We will do this using Foursquare

CLIENT_ID = 'SEB12VRXXLTFKQYNSB1DM2LAZEHFK2TVS02TL20ZYWH3IMSN' # your Foursquare ID
CLIENT_SECRET = '3E3LQBASDCDZAZOMI4JJRSLVFG45JOXXMFPIHKG1FF1ZM3Z4' # your Foursquare Secret
ACCESS_TOKEN = 'KSSMIWCOS5KDN4UV4KBSAAKZC2XOMHQEEWP01IAS4M2HYIH1' # your FourSquare Access Token
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

## Now let's identify the top 100 venues around each Attraction

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
                
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
sac_venues = getNearbyVenues(names=Attractions['Attraction'], 
                                 latitudes=Attractions['Latitude'], 
                                 longitudes=Attractions['Longitude'], 
                                 radius=500)

In [ ]:
# We identified 549 venues are within 500 feet from all 13 Attractions!
sac_venues.shape

In [ ]:
sac_venues.head()

In [ ]:
print('There are {} uniques categories of venues.'.format(len(sac_venues['Venue Category'].unique())))

## Lets get a count of how many venues are within 500 feet of each Attraction:

- There looks to be many venues by Capital Park, the Cathedral, in Old Sac and by the Railroad Museum
- The Caliornia Automobile Museum, the Crocker Museum and the American River Bike Trail seem to be more isolated with less venues

In [ ]:
sac_venues.groupby(['Neighborhood']).count()

## Let's analyse City of Sacramento Attractions and Venues even further!
- First, I will identify the Top 5 most frequented venue categories around each Attraction
- Then we can discover the top 10 most popular venues around each Attraction
- Finally, we can use K-means clustering to map out Attraction groupings to get more insight on the area

### With more insight, we can make a more informed decision on new Pizzeria location!

In [ ]:
# First, lets identify the top 5 venue categories for each Attraction using one hot encoding
sac_onehot = pd.get_dummies(sac_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
sac_onehot['Neighborhood'] =sac_venues['Neighborhood'] 

sac_onehot.shape

In [ ]:
sac_onehot.head()

In [ ]:
#Find the Neighborhood column
sac_onehot.columns[80]

In [ ]:
# move neighborhood column to the first column
fixed_columns = [sac_onehot.columns[80]] + list(sac_onehot.columns[0:80]) + list(sac_onehot.columns[81:])
sac_onehot = sac_onehot[fixed_columns]

sac_onehot.head()

In [ ]:
#make sure i have the same number of rows and columns (494, 151)
sac_onehot.shape

In [ ]:
# get the mean value of each category around an Attraction
sac_grouped = sac_onehot.groupby('Neighborhood').mean().reset_index()
sac_grouped.head()

In [ ]:
sac_grouped.shape

## Top 5 Most Popular Venue Categories for each Attraction location!

In [ ]:
# identify the most frequent categories of venues in each Neighborhood
num_top_venues = 5

for hood in sac_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = sac_grouped[sac_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

## Time to identify the top 10 most popular VENUES around each of the Attractions!

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = sac_grouped['Neighborhood']

for ind in np.arange(sac_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(sac_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.rename(columns={"Neighborhood":"Attraction"}, inplace=True)
neighborhoods_venues_sorted.head()

## Let's use K-means Clustering algorithm to create correlated clusters/groupings to gain new insight

In [ ]:
# set number of clusters
kclusters = 5

sac_grouped_clustering = sac_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(sac_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

In [ ]:
sac_grouped.head()

In [ ]:
sac_grouped_clustering.head()

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

sac_merged = Attractions



In [ ]:
sac_merged.head()

In [ ]:
neighborhoods_venues_sorted

In [ ]:
neighborhoods_venues_sorted.head()

In [ ]:
# merge sac_grouped with sac data to add latitude/longitude for each neighborhood
sac_merged = sac_merged.join(neighborhoods_venues_sorted.set_index('Attraction'), on='Attraction')

sac_merged.head(40)   # check the last columns!

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(sac_merged['Latitude'], sac_merged['Longitude'], sac_merged['Attraction'], sac_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=7,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Cluster 1 California Automobile Museum Area:
- A Gym, A Cycle studio, a handful of shops
- Close to the freeway intersection which may be convenient for delivery services.
- Very close to Old Sacramento on the water, super popular among visitors and locals!

In [ ]:
sac_merged.loc[sac_merged['Cluster Labels'] == 0, sac_merged.columns[[0] + list(range(4, sac_merged.shape[1]))]]


### Cluster 2 Capital Area, Coffee Shops, Resaurants, Bars, Pizzerias:
- This area is close to our State Capital, which is always hustling with California lawmakers and businesses.
- Definitely would be busy at lunch time catering to the State government businesses
- A Pizzeria in this area would benefit from high traffic, however there would be competing Pizzerias in this cluster
- There are many different restaurant choices in this area as well, lots of competition for that traffic!

In [ ]:
sac_merged.loc[sac_merged['Cluster Labels'] == 1, sac_merged.columns[[0] + list(range(4, sac_merged.shape[1]))]]

### Cluster 3 Sac Zoo Area, Theme Park, Shops and Recreation:
- This cluster is far from Sacramento City Center, so the foot traffic wouldn't be as high
- Recreational Lake and Golf Course and just a few shops and restaurants
- Patrons tend to eat while inside the Zoo or Theme Park
- May not be the best fit for a new Pizzeria

In [ ]:
sac_merged.loc[sac_merged['Cluster Labels'] == 2, sac_merged.columns[[0] + list(range(4, sac_merged.shape[1]))]]

### Cluster 4 Bike Trail Area, Recreation, Restaurants and Shops:
- This area is great for outdoor recreation
- A few shops and restaurants, and no popular Pizzerias
- Foot traffic would be lighter here than in the city center

In [ ]:
sac_merged.loc[sac_merged['Cluster Labels'] == 3, sac_merged.columns[[0] + list(range(4, sac_merged.shape[1]))]]

### Cluster 5 Crocker Museum Area, Coffee Shops, Restaurants, Shops & Park:
- There isnt a popular Pizzeria in the immediate area according to my analysis
- This is near highway, which could be convenient for Pizza delivery services
- The Park and Farmers Market is always a big draw

In [ ]:
sac_merged.loc[sac_merged['Cluster Labels'] == 4, sac_merged.columns[[0] + list(range(4, sac_merged.shape[1]))]]



# Part 2 Explore Cities on the OUTSKIRTS of Sacramento


- Looking at U.S. Consensus data for top 20 most populated cities near Sacramento
- Comparing each of the cities population and median family income to see if there is any correlation

In [ ]:
# The code was removed by Watson Studio for sharing.

In [ ]:
fig, ax = pyplot.subplots(figsize=(12,10))
plot=sns.set_style("whitegrid")
sns.set_context("paper", rc={"font.size":20, "axes.titlesize":30, "axes.labelsize":20})
#sns.set(rc={'figure.figsize':(11.7,8.27),"font.size":20,"axes.titlesize":20,"axes.labelsize":20},style="white")
ax.set_title("Median Family Income Boxplot")
ax=sns.boxplot(x=df["Median_Income"])
ax.set(xlabel="Median Family Income")
ax.set(ylabel="Min, Middle, Max Values")


In [ ]:
fig, ax = pyplot.subplots(figsize=(12,10))
plot=sns.set_style("whitegrid")
sns.set_context("paper", rc={"font.size":20,"axes.titlesize":30,"axes.labelsize":20})
#sns.set(rc={'figure.figsize':(11.7,8.27),"font.size":20,"axes.titlesize":20,"axes.labelsize":20},style="white")
ax.set_title("Population Boxplot")
ax=sns.boxplot(x=df["Population"])
ax.set(xlabel="Population")
ax.set(ylabel="Min, Middle, Max Values")

In [ ]:
fig, ax = pyplot.subplots(figsize=(12,10))
df['Population'].plot(kind='hist')
df['Population'].plot.hist()
ax.set_title("Population Histogram")
ax.set(xlabel="Population")
ax.set(ylabel="# of Cities")


In [ ]:
fig, ax = pyplot.subplots(figsize=(12,10))
df['Median_Income'].plot(kind='hist')
df['Median_Income'].plot.hist()
ax.set_title("Median Family Income Histogram")
ax.set(xlabel="Median Family Income")
ax.set(ylabel="# of Cities")



In [ ]:
fig, ax = pyplot.subplots(figsize=(12,10))
plot=sns.regplot(x="Population", y='Median_Income', data=df)
ax.set_title("Population:Income Scatterplot and Line of Regression")
ax.set(xlabel="Population")
ax.set(ylabel="Median Family Income")

### Analysis of Population and Median Family Income:

- The Regression Plot on Population vs Median_Income suggests that there is a positive correlation between the two variables
- This means that the cities with a larger population, also have a larger median family income
- If you choose to build your Pizzeria in the outskirts of Sacramento, I suggest choosing one with a larger population.
- Let's chose the top 5 highest populated cities outside sacramento and explore them more closely...

# Top 5 largest populated cities to explore: 
1. Elk Grove...............(pop. 174,775)
2. Roseville................(pop. 141,500)
3. Citrus Heights........(pop... 87,796)
3. Folsom...................(pop... 81,328)
4. Rancho Cordova....(pop... 75,087)


## Explorations:
- First, I will identify the Top 5 most frequented venue categories around each City
- Then we can discover the top 10 most popular venues around each City
- Finally, we can use K-means clustering to map out meaningful groupings to gain new insights

In [5]:

body = client_bc9e2ea9ed334541b53ed28bd585cd05.get_object(Bucket='applieddatasciencecapstoneproject-donotdelete-pr-c7ardu3bm1b4ka',Key='Cities.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df2 = pd.read_csv(body)
df2.head()


NameError: name 'client_bc9e2ea9ed334541b53ed28bd585cd05' is not defined

In [ ]:
names = df2.values.tolist()

In [ ]:
names

In [ ]:
# Used geolocator and used a loop to search for, intentify longitude and latitude, then populate a dataframe
Latitude2 = []
Longitude2 = []

for name in names:
    geolocator = Nominatim(user_agent="foursquare_agent")
    location = geolocator.geocode(name)
    latitude = location.latitude
    longitude = location.longitude
    print(f'{name}: [{latitude}, {longitude}]')
    Latitude2.append(latitude)
    Longitude2.append(longitude)        
        
print('\n')
print('DataFrames Latitude2 & Longitude2 have been updated.')

In [ ]:
Cities = pd.DataFrame({'City':names, 'Latitude':Latitude2, 'Longitude': Longitude2})
Cities

In [ ]:
Cities[0,0] = 'Elk Grove'   #df.ix['x','C']

#df2 = pd.DataFrame({'filename' :  'test2.dat', 'n':16}, index=[0])

In [ ]:
Cities

In [ ]:
Cities[1,0] = 'Roseville' 

In [ ]:
Cities[0,2] = 'Citrus Heights' 

In [ ]:
Cities[0,3] = 'Davis'

In [ ]:
Cities[0,4] = 'Folsom'

In [ ]:
Cities[0,5] = 'Rancho Cordova' 

In [ ]:
Cities[0,6] = 'Rocklin'

In [ ]:
Cities[0,7] = 'Woodland'

In [ ]:
Cities[0,8] = 'Yuba City'

In [ ]:
Cities[0,9] = 'Auburn'

In [ ]:
Cities[0,10] = 'Galt'

In [ ]:
Cities[0,11] = 'Grass Valley'

In [ ]:
Cities[0,12] = 'Lincoln'

In [ ]:
Cities[0,13] = 'Marysville'

In [ ]:
Cities[0,14] = 'Placerville'

In [ ]:
Cities[0,15] = 'South Lake Tahoe'

In [ ]:
Cities[0,16] = 'Truckee'

In [ ]:
Cities[0,17] = 'West Sacramento'

In [ ]:
Cities[0,18] = 'Live Oak'

In [ ]:
Cities[0,19] = 'Loomis'

In [ ]:
Cities